### Download the embeddings from wandb

In [3]:
import wandb
run = wandb.init()
artifact = run.use_artifact('bryars-bryars/cbow-wiki/embeddings:v4', type='embeddings')
artifact = run.use_artifact('bryars-bryars/cbow-wiki/model-weights:v6', type='model')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [10]:
import torch
from eve_cbow_model import CBOW

In [ ]:
# Load CBOW model
cbow = CBOW(vocab_size=30000, embedding_dim=200)
cbow.load_state_dict(torch.load(f"{artifact_dir}/model-weights:v6/cbow_epoch5_2025_04_17__15_28_43.pt"))
cbow.eval()

# Load embeddings if needed
cbow.embedding.weight.data = torch.load(f"{artifact_dir}/embeddings:v4/embeddings_epoch5_2025_04_17__15_28_43.pt")

# Setup regressor
mReg = Regressor(embedding_dim=128)
optimizer = torch.optim.Adam(mReg.parameters(), lr=0.005)

# Training loop
for i in range(100):
    trg = torch.tensor([[125.]], dtype=torch.float32)
    ipt = torch.tensor([[45, 27, 45367, 456]])

    with torch.no_grad():
        emb = cbow.embedding(ipt).mean(dim=1)

    out = mReg(emb)
    loss = torch.nn.functional.l1_loss(out, trg)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    wandb.log({'regression_loss': loss.item()})
    print(f"Step {i}: Loss = {loss.item():.4f}")
